# 03 — Interactive Dashboard (Plotly) — Integrated

In [ ]:
# Ensure deps in active kernel
import sys, subprocess, importlib
def ensure(pkg):
    try:
        importlib.import_module(pkg)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
ensure("pandas"); ensure("plotly")

In [ ]:
import pandas as pd, plotly.express as px
from pathlib import Path

ROOT = Path.cwd()
candidates = [ROOT.parent / "data/processed/fpds_cleaned.csv", ROOT / "data/processed/fpds_cleaned.csv"]
csv_path = next((p for p in candidates if p.exists()), None)
if csv_path is None:
    raise FileNotFoundError("Run 01_cleaning_fpds.ipynb first to create the processed file.")
df = pd.read_csv(csv_path, parse_dates=["action_date"])

### Interactive — Contract Volume by Month

In [ ]:
ts = df.groupby("year_month").size().reset_index(name="contracts").sort_values("year_month")
fig = px.line(ts, x="year_month", y="contracts", title="Contract Volume Over Time")
fig.update_layout(xaxis_title="Year-Month", yaxis_title="Contracts")
fig.show()

### Interactive — Average Turnaround Time by Month (BTS)

In [ ]:
if {"year_month","average_turnaround_time"}.issubset(df.columns):
    tt = df.groupby("year_month")["average_turnaround_time"].mean().reset_index()
    fig = px.line(tt, x="year_month", y="average_turnaround_time", title="Average Turnaround Time by Month (BTS)")
    fig.update_layout(xaxis_title="Year-Month", yaxis_title="Hours")
    fig.show()

### Interactive — Contracts by State vs. Percent On-Time (BTS)

In [ ]:
if {"state_usps","percent_on_time"}.issubset(df.columns):
    by_state = df.groupby("state_usps")["percent_on_time"].mean().reset_index()
    fig = px.bar(by_state, x="state_usps", y="percent_on_time", title="Average Percent On-Time by State (BTS)")
    fig.update_layout(xaxis_title="State", yaxis_title="% On-Time")
    fig.show()

### Interactive — Contracts by Sanctions Flag

In [ ]:
if "sanctioned_vendor" in df.columns:
    vc = df["sanctioned_vendor"].value_counts().reset_index()
    vc.columns = ["sanctioned_vendor","contracts"]
    fig = px.bar(vc, x="sanctioned_vendor", y="contracts", title="Contracts by Sanctioned Vendor Flag")
    fig.update_layout(xaxis_title="Sanctioned Vendor", yaxis_title="Contracts")
    fig.show()